In [12]:
import hvplot.pandas
import panel as pn
import holoviews as hv
import numpy as np
import pandas as pd
import os
import sys
import copy
sys.path.append(r"C:\Users\xavier.mouy\Documents\Workspace\GitHub\ecosound")
from ecosound.core.measurement import Measurement
from ecosound.core.audiotools import Sound
from ecosound.core.spectrogram import Spectrogram
from ecosound.visualization.grapher_builder import GrapherFactory
pn.extension()

# load dataset
data_file=r'C:\Users\xavier.mouy\Documents\PhD\Projects\Dectector\results\dataset_FS-NN.nc'
dataset = Measurement()
dataset.from_netcdf(data_file)
data_all = dataset.data
data = copy.deepcopy(data_all)
#data = data.sample(n = 10000)

# Widgets
inputs = list(data.columns)
X_variable  = pn.widgets.Select(name='X axis',
                                value='frequency_min',
                                options=inputs)
Y_variable  = pn.widgets.Select(name='Y axis',
                                value='frequency_max',
                                options=inputs)
Color  = pn.widgets.Select(name='Color',
                           value='label_class',
                           options=inputs)
alpha_slider = pn.widgets.FloatSlider(name='Point transparency',
                                      start=0, end=1,
                                      step=0.1,
                                      value=0.2,
                                     )
size_slider = pn.widgets.IntSlider(name='Point size',
                                   start=1,
                                   end=40,
                                   step=1,
                                   value=6,
                                  )

sampling_slider = pn.widgets.FloatSlider(name='Sampling ratio',
                                      start=0, end=1,
                                      step=0.1,
                                      value=0.2,
                                     )
# Stream
tap = hv.streams.Selection1D(index=[data.index.min()])

# Subsample
#@pn.depends(sampling_slider)
def subsample(data1, sampling_slider):
    n_points = int(len(data1)*sampling_slider)
    return data1.sample(n = n_points)

# Scatter plot
@pn.depends(X_variable, Y_variable, Color, alpha_slider, size_slider,sampling_slider)
def scatterplot(X_variable, Y_variable, Color, alpha_slider, size_slider,sampling_slider):
    global data
    data = subsample(data_all, sampling_slider)
    scatter_plot = data.hvplot.scatter(x=X_variable,
                                       y=Y_variable,
                                       c=Color,
                                       s=size_slider,
                                       alpha=alpha_slider,
                                       title= str(len(data)) + '/' + str(len(data_all)) + ' points',
                                       #datashade=True,
                                       tools=['tap'],
                                       active_tools=['wheel_zoom'],
                                      )
    tap.source = scatter_plot   
    #scatter_plot.opts(click_policy="hide")
    return scatter_plot   

# Measurement table
@pn.depends(tap.param.index)
def table(index):  
    if index:
        return pn.pane.DataFrame(data.iloc[index[0]])

# Spectrogram
@pn.depends(tap.param.index)
def spectrogram_plot(index):
    #data.iloc[index[0]]
    if index:
        frame = 0.0625 #3000
        nfft = 0.0853 # 4096
        step = 0.01 # 5
        fmin = 0
        fmax = 1000
        window_type = 'hann'
        time_buffer = 1
        palet = 'jet' # 'binary'

        data_selection = data.iloc[index[0]]

        wavfilename = os.path.join(data_selection.audio_file_dir, data_selection.audio_file_name + data_selection.audio_file_extension)
        t1 = data_selection.time_min_offset - time_buffer
        t2 = data_selection.time_max_offset + time_buffer
        # load audio data
        sound = Sound(wavfilename)
        sound.read(channel=0, chunk=[t1, t2], unit='sec', detrend=True)
        # Calculates  spectrogram
        spectro = Spectrogram(frame, window_type, nfft, step, sound.waveform_sampling_frequency, unit='sec')
        spectro.compute(sound, dB=True, use_dask=False, dask_chunks=40)

        # Plot
        graph = GrapherFactory('SoundPlotter', title=str(index[0]) + ': ' +data_selection.label_class + ' - ' +data_selection.label_subclass, frequency_max=fmax)
        graph.add_data(spectro)
        #graph.colormap = 'binary'
        graph.colormap = palet
        fig, ax = graph.show(display=False)
        return pn.pane.Matplotlib(fig)

# Dashboard
#gspec = pn.GridSpec(sizing_mode='stretch_both', max_height=800)
tabs = pn.Tabs(('Axes',pn.Column(X_variable, Y_variable, Color, sampling_slider)))
tabs.append(('Style',pn.Column(alpha_slider, size_slider)))
widgets = tabs
tabs2 = pn.Tabs(('Spectrogram',spectrogram_plot))
tabs2.append(('Measurements',table))

dashboard = pn.Column(pn.Row(widgets, scatterplot),tabs2)

dashboard.show()
#dashboard


C:\Users\xavier.mouy\Documents\Workspace\GitHub\ecosound\ecosound\visualization\sound_plotter.py:292: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  constrained_layout=True,


Launching server at http://localhost:59875


In [10]:
len(data)

11687

In [11]:
len(data_all)

58439

C:\Users\xavier.mouy\Documents\Workspace\GitHub\ecosound\ecosound\visualization\sound_plotter.py:292: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  constrained_layout=True,


C:\Users\xavier.mouy\Documents\Workspace\GitHub\ecosound\ecosound\visualization\sound_plotter.py:292: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  constrained_layout=True,


C:\Users\xavier.mouy\Documents\Workspace\GitHub\ecosound\ecosound\visualization\sound_plotter.py:292: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  constrained_layout=True,


C:\Users\xavier.mouy\Documents\Workspace\GitHub\ecosound\ecosound\visualization\sound_plotter.py:292: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  constrained_layout=True,


In [13]:
data

,date,uuid,from_detector,software_name,software_version,operator_name,UTC_offset,entry_date,audio_channel,audio_file_name,...,time_iqr,time_asymmetry,time_concentration,time_std,time_kurtosis,time_skewness,time_entropy,time_flatness,time_roughness,time_centroid
50052,2014-09-18 08:33:01.833,80f0722e-87f4-48fe-be5a-0637150ecf96,True,BlobDetector,0.1,xmouy-lpx,0.0,2020-06-08 11:39:33.709113,2,JASCOAMARHYDROPHONE742_20140918T083238.733Z,...,0.055,0.051429,0.067,0.034145,-1.337581,0.352902,-7.284042,0.978900,0.004554,0.089285
23134,2019-08-30 02:45:16.370,ad921e44-daa7-41c3-8ae5-e7dab92afb07,True,BlobDetector,0.1,xmouy-lpx,0.0,2020-06-05 17:08:11.433286,4,AMAR173.4.20190830T023532Z,...,0.071,-0.005464,0.080,0.021630,-1.287553,-0.370361,-7.495448,0.984221,0.003214,0.093362
39291,2019-09-16 04:38:45.400,9710bad7-b709-4d5c-9c0d-7f82b6530cda,True,BlobDetector,0.1,xmouy-lpx,0.0,2020-06-07 23:18:51.415925,4,AMAR173.4.20190916T041248Z,...,0.181,-0.031884,0.331,0.008879,0.303336,0.431974,-8.487562,0.993218,0.007477,0.179670
13488,2019-08-24 05:28:54.800,b25c2aff-c65a-40df-a49a-cef211dbf455,True,BlobDetector,0.1,xmouy-lpx,0.0,2020-06-05 16:58:50.582790,4,AMAR173.4.20190824T050532Z,...,0.020,0.000000,0.018,0.096847,-1.153628,0.609014,-5.889459,0.970199,0.004735,0.033926
13853,2019-08-24 05:32:33.880,05a88a7b-7a86-44af-bf44-939c721d7b01,True,BlobDetector,0.1,xmouy-lpx,0.0,2020-06-05 16:58:50.582790,4,AMAR173.4.20190824T050532Z,...,0.093,0.051163,0.107,0.015437,-1.082698,-0.354370,-7.797258,0.988678,0.003942,0.107877
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44684,2019-09-16 11:59:28.820,e21cd7bf-cc3e-4f16-ba6e-78959dbae658,True,BlobDetector,0.1,xmouy-lpx,0.0,2020-06-08 00:50:04.687609,4,AMAR173.4.20190916T114248Z,...,0.397,0.161905,0.674,0.004790,0.556213,1.019298,-9.561034,0.992862,0.004096,0.364807
10390,2019-08-20 20:42:47.990,ddfc5f49-c008-4bd3-8ed0-81482cab7fd6,True,BlobDetector,0.1,xmouy-lpx,0.0,2020-06-05 16:54:46.380895,4,AMAR173.4.20190820T203532Z,...,0.032,0.025000,0.030,0.038492,-1.189026,0.163562,-6.405803,0.989962,0.002032,0.040599
53557,2018-10-18 03:48:45.090,c928bcd8-fd46-4420-ab8c-3bbc12760dfb,True,BlobDetector,0.1,xmouy-lpx,-8.0,2020-06-08 07:53:21.751322,1,67674121.181018033806,...,0.059,0.023256,0.071,0.018134,-0.326553,-0.914644,-7.082610,0.993148,0.002172,0.064836
38934,2019-09-16 03:33:19.040,51b9ba3c-036e-4f7a-a5fd-d034d2cce41f,True,BlobDetector,0.1,xmouy-lpx,0.0,2020-06-07 23:09:39.834127,4,AMAR173.4.20190916T031248Z,...,0.030,0.042553,0.028,0.054767,-1.300859,0.527850,-6.525791,0.980349,0.003070,0.048406


C:\Users\xavier.mouy\Documents\Workspace\GitHub\ecosound\ecosound\visualization\sound_plotter.py:292: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  constrained_layout=True,


C:\Users\xavier.mouy\Documents\Workspace\GitHub\ecosound\ecosound\visualization\sound_plotter.py:292: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  constrained_layout=True,


C:\Users\xavier.mouy\Documents\Workspace\GitHub\ecosound\ecosound\visualization\sound_plotter.py:292: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  constrained_layout=True,


C:\Users\xavier.mouy\Documents\Workspace\GitHub\ecosound\ecosound\visualization\sound_plotter.py:292: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  constrained_layout=True,


...

C:\Users\xavier.mouy\Documents\Workspace\GitHub\ecosound\ecosound\visualization\sound_plotter.py:292: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  constrained_layout=True,


C:\Users\xavier.mouy\Documents\Workspace\GitHub\ecosound\ecosound\visualization\sound_plotter.py:292: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  constrained_layout=True,


C:\Users\xavier.mouy\Documents\Workspace\GitHub\ecosound\ecosound\visualization\sound_plotter.py:292: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  constrained_layout=True,


C:\Users\xavier.mouy\Documents\Workspace\GitHub\ecosound\ecosound\visualization\sound_plotter.py:292: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  constrained_layout=True,


C:\Users\xavier.mouy\Documents\Workspace\GitHub\ecosound\ecosound\visualization\sound_plotter.py:292: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  constrained_layout=True,
